# NYSE DataFrames and Databases

In this activity, you will read, clean and load data to a database table. 

## Instructions

### Create the Pandas DataFrame

1. Import the modules for Pandas, the Path, and SQLAlchemy.
2. Read the `nyse_companylist.csv` file from the Resources folder into a Pandas DataFrame called `nyse_df`. Review the DataFrame.
3. Check the data types (`dtypes`) of each column in the DataFrame.

### Clean the Pandas DataFrame

1. Apply the provided `clean_currency` function to the 'MarketCap' column of the `nyse_df` DataFrame.

  * The `clean_currency` function performs the following actions:
    -Removes non-numeric characters (ie '\$', 'M', and 'B') from the data entry
    -Converts the value to a float
    -Scales the data according to the suffixes 'M' and 'B'
    -Returns the cleaned and scaled data entry

2. Review the contents of the 'MarketCap' column to confirm that data has been cleaned as expected.

### Load the DataFrame into a SQLite Database

1. Create a database connection string that loads the SQLite database called `nyse.db` into the Resources folder.
2. Create a [database engine](https://docs.sqlalchemy.org/en/14/core/engines.html) called `nyse_engine`. Be sure to include the `echo=True` parameter.
3. Using the Pandas [`to_sql`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html) function, write the `nyse_df` DataFrame to a table called `NYSE` in the `nyse.db`. Be sure to include the parameters for `index` and `if_exists`.
4. Get the table name from the database to confirm it was created.

### Query the SQL Database

1. Write a SQL query to select all companies in the Technology sector AND in the Software industry AND that have a market cap greater than 30 billion USD.
2. Using the Pandas [`read_sql_query`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql_query.html) execute the SQL query and assign the results to a new Pandas DataFrame called `nyse_tech_companies_df`.
3. Review the DataFrame.



## Create the Pandas DataFrame

### 1. Import the modules for Pandas, the Path, and SQLAlchemy.

In [3]:
# Import modules
import pandas as pd
from pathlib import Path
import sqlalchemy as sql

### 2. Read the `nyse_companylist.csv` file from the Resources folder into a Pandas DataFrame called `nyse_df`. Review the DataFrame.

In [5]:
# Read the `nyse_companylist.csv` file from the Resources folder into a Pandas DataFrame 
nyse_df = pd.read_csv(
Path("../Resources/nyse_companylist.csv"))

# Review the DataFrame
nyse_df.head()


,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,Summary Quote,Unnamed: 8
0,DDD,3D Systems Corporation,4.79,$580.24M,NaN,Technology,Computer Software: Prepackaged Software,https://old.nasdaq.com/symbol/ddd,NaN
1,MMM,3M Company,161.66,$93.12B,NaN,Health Care,Medical/Dental Instruments,https://old.nasdaq.com/symbol/mmm,NaN
2,WBAI,500.com Limited,2.94,$126.42M,2013.0,Consumer Services,Services-Misc. Amusement & Recreation,https://old.nasdaq.com/symbol/wbai,NaN
3,EGHT,8x8 Inc,15.71,$1.64B,NaN,Technology,EDP Services,https://old.nasdaq.com/symbol/eght,NaN
4,AHC,A.H. Belo Corporation,1.39,$33.19M,NaN,Consumer Services,Newspapers/Magazines,https://old.nasdaq.com/symbol/ahc,NaN


### 3. Review the data types (`dtypes`) of each column in the DataFrame.

In [6]:
# Check dtypes of dataframe
nyse_df.dtypes


Symbol            object
Name              object
LastSale         float64
MarketCap         object
IPOyear          float64
Sector            object
industry          object
Summary Quote     object
Unnamed: 8       float64
dtype: object

## Clean the Pandas DataFrame

### 1. Apply the provided `clean_currency` function to the 'MarketCap' column of the `nyse_df` DataFrame. 

The `clean_currency` function performs the following actions:
   - Removes non-numeric characters (ie '$', 'M', and 'B') from the data entry
   - Converts the value to a float
   - Scales the data according to the suffixes 'M' and 'B'
   - Returns the cleaned and scaled data entry

In [7]:
# This function converts the string values into a floating point number
def clean_currency(price_string):
    price = price_string
    if type(price_string) == str:
        if price_string[-1] == 'M':
            million = 1000000
            price_string = price_string.replace('$', '')
            price_string = price_string.replace('M', '')
            price = float(price_string)
            price = price * million
        else:
            billion = 1000000000
            price_string = price_string.replace('$', '')
            price_string = price_string.replace('B', '')
            price = float(price_string)
            price = price * billion
    return price



In [11]:
# Apply the clean_currency function to MarketCap column
nyse_df['MarketCap'] = nyse_df['MarketCap'].apply(clean_currency)


### 2. Review the contents of the 'MarketCap' column.

In [12]:
# Review the contents for the MarketCap column
nyse_df['MarketCap']

0       5.802400e+08
1       9.312000e+10
2       1.264200e+08
3       1.640000e+09
4       3.319000e+07
            ...     
3173    2.822000e+10
3174    7.665000e+10
3175    2.506000e+10
3176    1.260000e+09
3177    2.050000e+09
Name: MarketCap, Length: 3178, dtype: float64

## Load the DataFrame into a SQLite Database

### 1. Create a database connection string that loads the SQLite database called `nyse.db` into the Resources folder.

In [13]:
# Create a database connection string
nyse_connection_string = 'sqlite:///../Resources/nyse.db'


### 2. Create a [database engine](https://docs.sqlalchemy.org/en/14/core/engines.html) called `nyse_engine`. Be sure to include the `echo=True` parameter.

In [14]:
# Create a database engine
nyse_engine = sql.create_engine(nyse_connection_string, echo=True)


### 3. Using the Pandas [`to_sql`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html) function, write the `nyse_df` DataFrame to a table called `NYSE` in the `nyse.db`.

In [16]:
# Add the cleaned data to a table named NYSE in the database.
nyse_df.to_sql('NYSE', nyse_engine, index=False, if_exists='replace')


2022-03-09 19:27:52,181 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("NYSE")
2022-03-09 19:27:52,182 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-03-09 19:27:52,184 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("NYSE")
2022-03-09 19:27:52,185 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-03-09 19:27:52,187 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-03-09 19:27:52,188 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-03-09 19:27:52,189 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("NYSE")
2022-03-09 19:27:52,190 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-03-09 19:27:52,191 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-03-09 19:27:52,192 INFO sqlalchemy.engine.Engine [raw sql] ('NYSE',)
2022-03-09 19:27:52,194 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("NYSE")
2022-03-09 19

### 4. Get the table name from the database to confirm it was created.

In [17]:
# Get the table name from the database
# YOUR CODE HERE!
nyse_engine.table_names()

2022-03-09 19:28:16,133 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-03-09 19:28:16,135 INFO sqlalchemy.engine.Engine [raw sql] ()


/Users/pete/opt/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:3: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  This is separate from the ipykernel package so we can avoid doing imports until


['NYSE']

## Query the SQL Database

### 1. Write a SQL query to select all companies in the Technology sector AND in the Software industry AND that have a market cap greater than 30 billion USD.

In [45]:
# Select all stocks where the industry is related to Software in the Technology sector having a market cap greater than 30,000,000,000.
sql_query = """
SELECT 
    * 
FROM NYSE
WHERE 
    Sector = 'Technology' 
  and industry LIKE '%Software%'
  and MarketCap > 30000000000
"""

### 2. Using the Pandas [`read_sql_query`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql_query.html) execute the SQL query and assign the results to a new Pandas DataFrame called `nyse_tech_companies_df`.

In [43]:
# Execute the SQL query and read the data into a Pandas DataFrame
nyse_tech_companies_df = nyse_tech_companies_df = pd.read_sql_query(sql_query, nyse_engine, index_col='Symbol')


2022-03-09 19:39:24,348 INFO sqlalchemy.engine.Engine 
SELECT 
    * 
FROM NYSE
WHERE 
    Sector = 'Technology' 
  and industry like '%Software%'
  and MarketCap > 30000000000

2022-03-09 19:39:24,354 INFO sqlalchemy.engine.Engine [raw sql] ()


### 3. Review the DataFrame.

In [44]:
# Review the DataFrame
# YOUR CODE HERE!
nyse_tech_companies_df

,Name,LastSale,MarketCap,IPOyear,Sector,industry,Summary Quote,Unnamed: 8
Symbol,,,,,,,,
INFO,IHS Markit Ltd.,80.06,3.177000e+10,1996.0,Technology,"Computer Software: Programming, Data Processing",https://old.nasdaq.com/symbol/info,None
ORCL,Oracle Corporation,59.58,1.793900e+11,1986.0,Technology,Computer Software: Prepackaged Software,https://old.nasdaq.com/symbol/orcl,None
CRM,Salesforce.com Inc,246.67,2.244700e+11,2004.0,Technology,Computer Software: Prepackaged Software,https://old.nasdaq.com/symbol/crm,None
SAP,SAP SE,157.08,1.875000e+11,NaN,Technology,Computer Software: Prepackaged Software,https://old.nasdaq.com/symbol/sap,None
NOW,"ServiceNow, Inc.",484.21,9.287000e+10,2012.0,Technology,Computer Software: Prepackaged Software,https://old.nasdaq.com/symbol/now,None
SHOP,Shopify Inc.,966.50,1.177900e+11,2015.0,Technology,Computer Software: Prepackaged Software,https://old.nasdaq.com/symbol/shop,None
SNAP,Snap Inc.,25.74,3.782000e+10,2017.0,Technology,"Computer Software: Programming, Data Processing",https://old.nasdaq.com/symbol/snap,None
SNOW,Snowflake Inc.,250.41,6.929000e+10,2020.0,Technology,Computer Software: Prepackaged Software,https://old.nasdaq.com/symbol/snow,None
SQ,"Square, Inc.",160.76,7.130000e+10,2015.0,Technology,Computer Software: Prepackaged Software,https://old.nasdaq.com/symbol/sq,None
